<a href="https://colab.research.google.com/github/ozloty06/PySpark_Amazon-Vine-Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.2 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-15 01:13:46--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.03MB/s    in 0.2s    

2021-08-15 01:13:47 (6.03 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Vine-Challange").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
# Extract the dataset used in Deliverable 1
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [6]:
# Recreate vine table and analysis from Deliverable 1
# Clean up tables
dropna_df=df.dropna()
dropna_df=df.distinct()
dropna_df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26197704|R10010D6B1QM7U|B004HX5OFW|       8155204|Xbox 360 Slim AC ...|     Video Games|          1|            0|          0|   N|                Y|                 POS|I purchased this ...| 2013-03-28|
|         US|   10409381|R100BI61LIKH2N|B000QW9D14|     583724133|Blue Dragon - Xbo...|     Video Games|          5|    

In [7]:
dropna_df.count()

1785997

In [8]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
dropna_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26197704|R10010D6B1QM7U|B004HX5OFW|       8155204|Xbox 360 Slim AC ...|     Video Games|          1|            0|          0|   N|                Y|                 POS|I purchased this ...| 2013-03-28|
|         US|   10409381|R100BI61LIKH2N|B000QW9D14|     583724133|Blue Dragon - Xbo...|     Video Games|          5|    

In [9]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = dropna_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(15)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R10010D6B1QM7U|          1|            0|          0|   N|                Y|
|R100BI61LIKH2N|          5|            1|          2|   N|                Y|
|R100TPJCYDYGBU|          3|            1|          2|   N|                Y|
|R1018OO9U6UUWO|          5|            0|          0|   N|                Y|
|R102YIWZEIAXT9|          5|            2|          4|   N|                N|
|R1034S7FA21OI1|          5|            2|          2|   N|                Y|
|R103LC8CTAHWZ6|          2|            1|          1|   N|                Y|
|R103PT7S4HRIP8|          2|            0|          0|   N|                Y|
|R104RCQZYPYDXS|          5|            7|          8|   N|                N|
|R105KZIXHZW63W|          5|            1|          1|   N|     

In [10]:
# Step 1: Filter/create DataFrame where total_votes count >= 20 
vine_total_votes_df= vine_df.filter("total_votes>=20").select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_total_votes_df.show(15)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y|
|R2NPZ79NDZ87XO|          1|            1|         31|   N|                N|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N|
|R13GZAFU085VG6|          4|            8|         41|   N|                Y|
|R3VNNA3YL48039|          5|          181|        200|   N|                Y|
|R3P830AA62XHXA|          1|            8|         20|   N|                N|
| RZAE1OYOXEOAR|          5|           86|         94|   N|                Y|
|R1TBFM7966BSCZ|          1|           33|         56|   N|     

In [23]:
# check datatypes
# vine_total_votes_df.dtypes
# Step 2: Filter/create DataFrame where (helpful_votes/total_votes) => 50%
from pyspark.sql.functions import col
vine_percents_df=vine_total_votes_df.withColumn('percent', (col("helpful_votes")/ col("total_votes"))*100)
vine_percents_df=vine_percents_df.filter('percent>=0.5')
vine_percents_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y| 91.48936170212765|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y| 93.47826086956522|
|R2NPZ79NDZ87XO|          1|            1|         31|   N|                N| 3.225806451612903|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y| 83.63636363636363|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N| 57.14285714285714|
|R13GZAFU085VG6|          4|            8|         41|   N|                Y| 19.51219512195122|
|R3VNNA3YL48039|          5|          181|        200|   N|                Y|              90.5|
|R3P830AA62XHXA|          1|  

In [33]:
# Step 3: Filter/create "paid" DataFrame where vine == 'Y'
vine_Y_df=vine_percents_df.filter(col("vine").contains("Y"))
vine_Y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R24PS9N276OXF4|          5|          241|        262|   Y|                N| 91.98473282442748|
|R3EPHK0JGQGE91|          5|           21|         28|   Y|                N|              75.0|
|R3T4LTJ189G0LT|          4|           37|         45|   Y|                N| 82.22222222222221|
| RKG407AVAZXBQ|          3|           22|         25|   Y|                N|              88.0|
| RNU8PK609WT6P|          4|          347|        362|   Y|                N| 95.85635359116023|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N| 76.92307692307693|
|R23WY68NT94WKE|          5|          125|        143|   Y|                N| 87.41258741258741|
|R3JT907UUA1WUI|          4|  

In [34]:
# Step 4: Filter/create "unpaid" DataFrame where vine == 'N'
vine_N_df=vine_percents_df.filter(col("vine").contains("N"))
vine_N_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y| 91.48936170212765|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y| 93.47826086956522|
|R2NPZ79NDZ87XO|          1|            1|         31|   N|                N| 3.225806451612903|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y| 83.63636363636363|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N| 57.14285714285714|
|R13GZAFU085VG6|          4|            8|         41|   N|                Y| 19.51219512195122|
|R3VNNA3YL48039|          5|          181|        200|   N|                Y|              90.5|
|R3P830AA62XHXA|          1|  

In [59]:
# Count helpful votes in non-vine
vine_N_df.filter("helpful_votes>100").count()

3091

In [63]:
# What percent of non-vine votes are helpful
3091/64000*100

4.8296875

In [60]:
# Count helpful votes in vine
vine_Y_df.filter("helpful_votes>100").count()

9

In [64]:
# What percent of vine votes are helpful
9/102*100

8.823529411764707

In [35]:
# Count verified purchase outside of vine program
vine_verified=vine_N_df.filter(col("verified_purchase").contains("Y"))
vine_verified.count()

15260

In [36]:
# Count verified purchases with 5 star rating outside of vine program
vine_verified=vine_N_df.filter(col("verified_purchase").contains("Y"))
vine_verified.filter(vine_verified.star_rating == 5).count()

4660

In [58]:
# Percent of non-vine verified purchases with a 5 star rating
4660/15260*100

30.53735255570118

In [37]:
# Step 5: Determine total reviews, 5-star reviews, and percentage of 5-star reviews for both Dataframes
# 5a: count of all rows
vine_Y_df.count()

102

In [38]:
# 5a: count of all rows
vine_N_df.count()

64000

In [39]:
# 5b: count of star_rating = 5
vine_Y_df.filter(vine_Y_df.star_rating == 5).count()

48

In [40]:
# 5b: count of star_rating = 5
vine_N_df.filter(vine_N_df.star_rating == 5).count()

20282

In [41]:
# 5c: percent of all rows with a 5 star_rating
(vine_Y_df.filter(vine_Y_df.star_rating == 5).count())/(vine_Y_df.count())*100

47.05882352941176

In [42]:
# 5c: percent of all rows with a 5 star_rating
(vine_N_df.filter(vine_N_df.star_rating == 5).count())/(vine_N_df.count())*100

31.690625

In [57]:
# review count difference 
((102/(64000+102))*100)

0.15912140026832236

In [47]:
# percent difference between star ratings for vine vs non-vine
(vine_Y_df.filter(vine_Y_df.star_rating == 5).count())/((vine_Y_df.count())-(vine_N_df.filter(vine_N_df.star_rating == 5).count())/(vine_N_df.count()))*100

47.20548739204741

In [53]:
final_df = spark.createDataFrame(
    [
        (1, "Vine_Paid", "102", "48", "47%"),  
        (2, "Non-Vine_Unpaid", "64,000", "20,282", "31%"),
    ],
    ["id", "Program", "Total_Reviews", "5_Star_Reviews", "Percent_5_Star"]
)

final_df.show()

+---+---------------+-------------+--------------+--------------+
| id|        Program|Total_Reviews|5_Star_Reviews|Percent_5_Star|
+---+---------------+-------------+--------------+--------------+
|  1|      Vine_Paid|          102|            48|           47%|
|  2|Non-Vine_Unpaid|       64,000|        20,282|           31%|
+---+---------------+-------------+--------------+--------------+

